## <span style="color: green;">**Llama2.3 - 3B**</span> as our <span style="color: red;">**RAG**</span> model to generate and reforamt our response from our Vektor_DB 

In [3]:
!pip install accelerate # Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate

  Using cached accelerate-1.2.1-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-1.2.1-py3-none-any.whl (336 kB)


### Retrieve Data from our VektorDatavase

In [1]:
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import chromadb


db_path="./vektor_DB"
client = chromadb.PersistentClient(path=db_path)

embedding_model = SentenceTransformer("sentence-transformers/gtr-t5-large")

collection = client.get_collection("meinungen")

def query_collection(query_text, n_results=4):

    query_embedding = embedding_model.encode(query_text)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results,
        include=["documents", "metadatas", "distances"]
    )
    return results

d:\Projects\UNI\KI und Big Data\KI-BigData\tst\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Initialize the Llama-3B pipeline

In [2]:
from transformers import pipeline

rag_pipeline = pipeline(
    "text-generation",
    model="NousResearch/Hermes-3-Llama-3.2-3B",
    device_map="auto",  # I am using my GPU, but I did make it as Auto - if my gpu not available it will use the cpu
    torch_dtype="auto" # Use the most efficient data type for the hardware
)


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.02s/it]
Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


### Without prompt instructions

In [ ]:

def query_with_llama(query_text):
    results = query_collection(query_text)
    context = ""
    for i in range(len(results["documents"][0])):
        context += f"Document {i+1}:\n"
        context += f"{results['documents'][0][i]}\n"
        context += f"Party: {results['metadatas'][0][i]['party']}\n"


    prompt = f"""
    You are a political AI assistant.  Based on the following information retrieved from a vector database, answer the user's question.  If the provided data is not relevant to the question, respond with "I do not know".

    User Question: {query_text}

    Retrieved Data:
    {context}

    Your Answer:
    """

    generated_text = rag_pipeline(prompt, max_new_tokens=250)[0]["generated_text"]
    # Extract the answer from the generated text (remove the prompt)
    answer = generated_text[len(prompt):].strip()
    return answer


query = "Was denkt die AFD über Islamunterricht an deutschen Schulen?"
answer = query_with_llama(query)
print(answer)

Die AFD unterstützt die Einführung eines deutschsprachigen Islamunterrichts unter staatlicher Aufsicht. Dabei sollten die Lehrer*innen an deutschen Hochschulen ausgebildet werden und Imame einer islamischen Religionsgemeinschaft an deutschen Hochschulen ausgebildet werden, um diese auszuwählen. Die AFD ist der Meinung, dass der Islam mit der freiheitlich-demokratischen Grundordnung Deutschlands nicht vereinbar ist, wenn er einen Herrschaftsanspruch als alleingültige Religion erhebt und unsere Rechtsordnung nicht voll anerkennt oder bekämpft. Die AfD fordert, dass Imame in Deutschland möglichst in deutscher Sprache predigen und ein Zertifikat B2 für die deutsche Sprache des Gemeinsamen Europäischen Referenzrahmens für Sprachen vorweisen müssen. Die AfD lehnt es ab, die Verleihung des Status als Körperschaft öffentlichen Rechts an islamische Organisationen zu genehmigen. Die AfD möchte, dass im konfessionsgebundenen Religionsunterricht an staatlichen


### Construct the prompt

In [3]:
def clean_prompt(query_text, results):
    if not results["documents"] or not results["documents"][0]:
        return None  # Return None if no data is retrieved

    # Construct the context
    context = "\n".join(
        f"Document {i + 1}:\n{doc}\nParty: {results['metadatas'][0][i]['party']}"
        for i, doc in enumerate(results["documents"][0])
    )
    prompt = f"""
You are a political AI assistant. Based on the following information retrieved from a vector database, answer the user's question. If the provided data is not relevant to the question, respond with "I do not know".

User Question: {query_text}

Retrieved Data:
{context}

Your Answer:
"""
    return prompt

###  Retrieves data from the vector database and queries the AI model with iterative generation to handle truncation.

In [6]:
def query_with_llama(query_text):
    # call the Retrieve function
    results = query_collection(query_text)

    # call the prompt Constructor
    prompt = clean_prompt(query_text, results)
    if not prompt:
        return "I do not know."

    generated_text = ""
    stop_signal = "\n\n"
    is_complete = False

    while not is_complete:
        response = rag_pipeline(prompt + generated_text, max_new_tokens=300)[0]["generated_text"]

        new_text = response[len(prompt) + len(generated_text):].strip()
        generated_text += new_text

        if stop_signal in new_text or len(new_text) < 300:
            is_complete = True

    # Extract the final answer
    answer = generated_text.strip()
    return answer

### Try it

In [9]:
query = "Was denkt die AFD über Islamunterricht an deutschen Schulen?"
answer = query_with_llama(query)
print(answer)

Die AfD befürwortet den Islamunterricht in Deutschland nur unter bestimmten Bedingungen. Sie fordert eine sachliche Islamkunde im Ethikunterricht und nicht einen Religionsunterricht, der als "bekenntnisgebunden" gewertet wird. Die Lehrer*innen sollen an deutschen Hochschulen ausgebildet werden und Imame müssen eine solche Ausbildung haben, um eine Zulassung zu erhalten. Zudem sollte der Islamunterricht in deutscher Sprache stattfinden und die Predigenden Imame sich zu unserer Verfassung bekennen. Die AFD ist sich darüber im Klaren, dass der Islam in Deutschland mit der freiheitlich-demokratischen Grundordnung nicht vereinbar ist, wenn er Herrschaftsansprüche als alleingültige Religion erhebt und unsere Rechtsordnung bekämpft.


### Improve the prompt more and check if the question contains one of listed parties

In [3]:
def imp_clean_prompt(query_text, results):
    if not results["documents"] or not results["documents"][0]:
        return None

    # List of parties to check
    parties = ["AFD", "Die Linke", "FreieWahler", "SPD", "FDP", "Grünen", "CDU CSU"]
    
    mentioned_parties = [party for party in parties if party.lower() in query_text.lower()]
    

    party_docs = {}
    for i, doc in enumerate(results["documents"][0]):
        party = results['metadatas'][0][i]['party']
        if party not in party_docs:
            party_docs[party] = []
        party_docs[party].append(f"Document {i + 1}:\n{doc}")
    
    # If a party is mentioned in the query, only include its documents
    if mentioned_parties:
        context_parts = []
        for party in mentioned_parties:
            if party in party_docs:
                context_parts.append(f"Party: {party}")
                context_parts.extend(party_docs[party])
    else:
        # If not, include all
        context_parts = []
        for party, docs in sorted(party_docs.items()):
            context_parts.append(f"Party: {party}")
            context_parts.extend(docs)

    context = "\n\n".join(context_parts)

    # I wanted to see what I have done here 🤷‍♂️
    print(context)

    prompt = f"""
You are a political AI assistant. Based on the following information retrieved from a vector database, answer the user's question. If the provided data is not relevant to the question, respond with "I do not know".

User Question: {query_text}

Retrieved Data:
{context}

Your Answer:
"""
    return prompt


## Let us try it again 🤣

In [5]:
def query_with_llama(query_text):
    # call the Retrieve function
    results = query_collection(query_text)

    # call the new prompt Constructor
    prompt = imp_clean_prompt(query_text, results)
    if not prompt:
        return "I do not know."

    generated_text = ""
    stop_signal = "\n\n"
    is_complete = False

    while not is_complete:
        response = rag_pipeline(prompt + generated_text, max_new_tokens=270)[0]["generated_text"]

        new_text = response[len(prompt) + len(generated_text):].strip()
        generated_text += new_text

        if stop_signal in new_text or len(new_text) < 270:
            is_complete = True

    # Extract the final answer
    answer = generated_text.strip()
    return answer

query = "Was denkt die AFD über Islamunterricht an deutschen Schulen?"
answer = query_with_llama(query)
print(answer)

Party: AFD

Document 1:
Islamunterricht an deutschen Schulen dient derzeit nicht der Integration, sondern fördert ein Verharren in Einstellungen, die echte Integration verhindern. Da die islamischen Gemeinschaften in Deutschland keine kirchenähnliche Struktur aufweisen, kann ihnen das Privileg eines „bekenntnisgebundenen“ Religionsunterrichts an staatlichen Schulen nicht zugestanden werden

Document 3:
Die AfD steht uneingeschränkt zur Glaubens-, Gewissens- und Bekenntnisfreiheit nach Art. 4 Grundgesetz. Muslime, die sich integrieren und unsere Grundordnung und die Grundrechte anerkennen, sind geschätzte Mitglieder unserer Gesellschaft. In unserem säkularen Staat steht staatliches Recht über religiösen Geboten. Soweit der Islam einen Herrschaftsanspruch als alleingültige Religion erhebt, die Rechtsgrundsätze der Scharia auch in Deutschland durchsetzen will und unsere Rechtsordnung nicht voll anerkennt oder sogar bekämpft, ist er mit der freiheitlich-demokratischen Grundordnung Deutschl